In [1]:
import os
import numpy as np
from re import search

# from geopandas import datasets, GeoDataFrame, read_file
import pandas as pd
import geopandas as gpd

import datetime
import calendar

import warnings
warnings.filterwarnings('ignore')

In [2]:
def ConditionDaysGetMonth(x):
    m = int(round(x/30.08))
    if x < 30:
        m = 0
    if x > 365 and x <= 365+15:
        m = 12
    if x > 365+15:
        m = -9999
    return int(m)
def agem45(startdate, enddate):
    if startdate == None or startdate == 'None':
        agemonth = None
    else:
        year = startdate.split('-')[0]
        month = startdate.split('-')[1]
        day = startdate.split('-')[2]

        yearp = enddate.split('-')[0]
        monthp = enddate.split('-')[1]
        dayp = enddate.split('-')[2]

        start_date = datetime.date(int(year), int(month), int(day))
        end_date = datetime.date(int(yearp), int(monthp), int(dayp))

        time_difference = end_date - start_date
    #             print(time_difference.days)

        agemonth = ConditionDaysGetMonth(int(time_difference.days))
    
    return agemonth

def formatdate(date):
    return date[:4]+'-'+date[4:6]+'-'+date[6:]

def last_date_of_month(date):
    yyyy = date.split('-')[0]
    mm = date.split('-')[1]
    _, last_day = calendar.monthrange(int(yyyy), int(mm))
    
    last_date = date.replace(date[-2:], str(last_day))
    return last_date

def stantdarddicts(excelfile, sheet):
    xls = pd.read_excel(excelfile, sheet_name=sheet) 
    dicts = {}
    for item_ptype in ptype:
        age = {}
        for i in range(len(xls)):
        #     print(i)
            if xls[xls.columns[0]][i] == item_ptype:
                for j in range(len(monthcols)):
                    ndviavg =xls[monthcols[j]][i]
                    age[j+1] = ndviavg
    #                 print(item_ptype,j+1,xls[monthcols[j]][i])
        dicts[item_ptype] = age

    return dicts 

def calculatepercentagedif(ptype,age,ndvi,stdicts):
    result = None
    if ptype != 'None' and  age > 0 and ndvi != 'None':     
        standardndvi = stdicts[ptype][int(age)]
        if str(standardndvi) != 'None':
            result = ((float(ndvi)-standardndvi)/standardndvi)*100  
    return result

In [3]:
excelfile = '/crophealth/standardline/STLine_2023.xlsx' #Main , RES, Yield => >20:NP, >18RT
rfile = '/crophealth/temp/NewPlantdateFromTimeseries/MPK_20230716_PLOT_PN_VersionUpgrade.geojson'

In [4]:
monthcols = ['m01','m02','m03','m04','m05','m06','m07','m08','m09','m10','m11','m12']
ptype = ['NP', 'RT']

In [5]:
gdf = gpd.read_file(rfile)

gdf['datenow'] = '2023-08-17'
gdf['Age'] = gdf.apply(lambda x: agem45(x.ch_plant_date_ff, x.datenow), axis=1)

cols = []  ### for create new         
colsmean = []
for item in gdf.columns:
    if len(item.split('_')) == 4:
        if (item.split('_')[-2] == '01' or item.split('_')[-2] == '16') and item.split('_')[-1] == 'mean':
            colsmean.append(item)
#             colsndvits.append(item)
        elif item.split('_')[-1] != 'percloud':
            cols.append(item)
    else:
        cols.append(item)

cols.append('NDVI')
cols.append('Cloudcover')

ndvilastcomposite = colsmean[-1]
cloudlastcomposite = colsmean[-1].replace('mean', 'percloud')
gdf['NDVI'] = gdf[ndvilastcomposite].astype('str')
gdf['Cloudcover'] = gdf[cloudlastcomposite]


# perdifversion = ['Main','RES','Yield','Current']
perdifversion = ['Main','RES','Yield']

for item in perdifversion:
    colname = 'perdif_'+item
    if item == 'Current':
        dicts = dicts_current
    else:
        dicts = stantdarddicts(excelfile, item)
    gdf[colname] = gdf.apply(lambda x: calculatepercentagedif(x.plot_type_ff, x.Age, x.NDVI, dicts), axis=1)
    cols.append(colname)


gdffile = gdf[cols]

gdff = gpd.GeoDataFrame(gdffile,  geometry='geometry')
output = '/crophealth/temp/CHM_MSPN_Dif_result.geojson'
gdff.to_crs(epsg=4326).to_file(output, driver='GeoJSON', encoding='utf-8')

print('--- Done-----')

--- Done-----


In [6]:
gdffile[(gdffile['perdif_Main'] > 0) | (gdffile['perdif_Main'] < 0)][['plot_code','plot_id','Age','NDVI','Cloudcover','perdif_Main','perdif_RES','perdif_Yield']]

,plot_code,plot_id,Age,NDVI,Cloudcover,perdif_Main,perdif_RES,perdif_Yield
3,1116450015,3,8.0,0.50107,0.0,-31.704526,-32.455875,-33.657567
6,1087150012,6,8.0,0.75166,0.0,2.450707,1.323602,-0.479069
31,1017220013,31,8.0,0.7093699999999999,0.0,-3.313389,-4.377081,-6.078330
50,1237800001,50,8.0,0.7837499999999999,0.0,6.824550,5.649327,3.769696
51,1134100004,51,8.0,0.77547,0.0,5.695992,4.533185,2.673411
...,...,...,...,...,...,...,...,...
38427,1171030010,38427,5.0,0.83694,0.0,20.853415,34.441568,24.591597
38449,1201260035,38449,7.0,0.78646,0.0,4.589988,14.657361,9.585020
38450,1145820029,38450,5.0,0.63288,0.0,-8.612674,1.662461,-5.785923
38453,1052240017,38453,6.0,0.7569,0.0,3.592158,13.429353,7.380723
